# IDS Project Notebook

Αυτό το notebook είναι starter για σύστημα ανίχνευσης εισβολών χρησιμοποιώντας το dataset `pcap_data.csv` από το GitHub. Περιλαμβάνει supervised και unsupervised προσέγγιση καθώς και βασική προεπεξεργασία.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt

# Για αποθήκευση
import os


In [ ]:
# 1. Φόρτωση δεδομένων από το GitHub (raw)
url = "https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/pcap_data.csv"
try:
    df = pd.read_csv(url)
except Exception as e:
    print('Σφάλμα φόρτωσης (πιθανόν λόγω αποσυνδεδεμένου περιβάλλοντος):', e)
    df = pd.DataFrame()

# 2. Προεπισκόπηση
print('Σχήμα dataset:', df.shape)
display(df.head())


In [ ]:
# 3. Απλή προεπεξεργασία
if not df.empty:
    # Έλεγχος για ελλείπουσες τιμές
    print('Ελλείπουσες τιμές ανά στήλη:')
    print(df.isnull().sum())

    # Διαχωρισμός αριθμητικών στηλών
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    print('Αριθμητικές στήλες:', numeric_cols)

    # Scaling των αριθμητικών χαρακτηριστικών
    scaler = StandardScaler()
    if numeric_cols:
        df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Αν υπάρχει ετικέτα, supervised
    if 'label' in df.columns:
        X = df.drop(columns=['label'])
        y = df['label']
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y if len(np.unique(y))>1 else None
        )
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print('Supervised classification report:')
        print(classification_report(y_test, y_pred))
        if hasattr(clf, 'predict_proba'):
            try:
                roc = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
                print('ROC AUC:', roc)
            except Exception as e:
                print('Δεν υπολογίστηκε ROC AUC:', e)
    else:
        # Unsupervised anomaly detection
        iso = IsolationForest(contamination=0.01, random_state=42)
        iso.fit(df[numeric_cols])
        anomalies = iso.predict(df[numeric_cols])  # -1 anomaly
        df['anomaly'] = (anomalies == -1).astype(int)
        print('Πλήθος ανωμαλιών:', df['anomaly'].sum())
else:
    print('Το dataset είναι άδειο, παρακαλώ ελέγξτε τη φόρτωση.')


In [ ]:
# 4. Αποθήκευση επεξεργασμένων / αποτελεσμάτων
output_path = 'IDS_processed_results.csv'
try:
    if not df.empty:
        df.to_csv(output_path, index=False)
        print(f'Αποθηκεύτηκαν αποτελέσματα σε {output_path}')
except Exception as e:
    print('Σφάλμα κατά την αποθήκευση:', e)


In [ ]:
# 5. Προτεινόμενα επόμενα βήματα
# - Προσθήκη Autoencoder με Keras για reconstruction-based anomaly detection.
# - Βελτίωση επιλογής χαρακτηριστικών (feature selection / importance).
# - Διαχωρισμός σε pipeline και χρήση GridSearchCV για tuning.
# - Οπτικοποίηση (confusion matrix heatmap, histograms reconstruction error).
